In [11]:
# import
import pandas as pd
import numpy as np
import json
from utils import min_mse
import plotly.express as px
import os


In [12]:
# constants

ALPHA = 1/137
PI = np.pi

m_p_n = 1.67 * (10**-24)


In [13]:
# input data
element = "C" # choose any
EeV = 6000

In [14]:
# data from element
path = "data/updated_periodic_table.csv"
print(path)
periodic_table = pd.read_csv(path, usecols=[1, 3, 4, 20, 29])

periodic_table.columns = ["Z", "Symbol", "N", "ro", "chi"]
periodic_table.columns
data = periodic_table[periodic_table["Symbol"] == element]

data/updated_periodic_table.csv


In [15]:
# vars
Z = data["Z"].iloc[0]
N = data["N"].iloc[0]
ro = data["ro"].iloc[0]
chi = data["chi"].iloc[0]
print("Z =", Z, "N =", N, "ro =", ro,"chi =", chi)

Z = 6 N = 12.011 ro = 2.27 chi = {'10.0': {'f1': -9999.0, 'f2': 0.806885}, '10.1617': {'f1': -9999.0, 'f2': 0.851522}, '10.3261': {'f1': -9999.0, 'f2': 0.898628}, '10.4931': {'f1': -9999.0, 'f2': 0.948341}, '10.6628': {'f1': -9999.0, 'f2': 1.0008}, '10.8353': {'f1': -9999.0, 'f2': 1.05755}, '11.0106': {'f1': -9999.0, 'f2': 1.12167}, '11.1886': {'f1': -9999.0, 'f2': 1.18968}, '11.3696': {'f1': -9999.0, 'f2': 1.26181}, '11.5535': {'f1': -9999.0, 'f2': 1.33832}, '11.7404': {'f1': -9999.0, 'f2': 1.41946}, '11.9303': {'f1': -9999.0, 'f2': 1.50553}, '12.1232': {'f1': -9999.0, 'f2': 1.58971}, '12.3193': {'f1': -9999.0, 'f2': 1.6782}, '12.5186': {'f1': -9999.0, 'f2': 1.77161}, '12.721': {'f1': -9999.0, 'f2': 1.87023}, '12.9268': {'f1': -9999.0, 'f2': 1.97433}, '13.1359': {'f1': -9999.0, 'f2': 2.08423}, '13.3483': {'f1': -9999.0, 'f2': 2.20025}, '13.5642': {'f1': -9999.0, 'f2': 2.29235}, '13.7836': {'f1': -9999.0, 'f2': 2.38287}, '14.0066': {'f1': -9999.0, 'f2': 2.47696}, '14.2331': {'f1': -999

In [16]:
# ather vars
m_e = 0.18 * (10**-24)
m_a = (N * m_p_n + Z * m_e)
n0_m = (ro / (5 * m_a))
k_n0_m = (n0_m * 10**4) / 125
om2 = ((4 * PI * Z) / 137) * k_n0_m
print("m_e = ", m_e, "m_a = ", m_a, "n0_m = ", n0_m, "k_n0_m = ", k_n0_m, "om2 = ", om2)



m_e =  1.7999999999999997e-25 m_a =  2.1138369999999994e-23 n0_m =  2.147753114360285e+22 k_n0_m =  1.718202491488228e+24 om2 =  9.456161736594038e+23


In [17]:
# get chi from JSON
chi_string = chi.replace("'", '"')
chi_dct = json.loads(chi_string)
dict_values = [float(i) for i in chi_dct]
dict_values[:5]

[10.0, 10.1617, 10.3261, 10.4931, 10.6628]

In [18]:
# vars
EeV_s = min_mse(EeV, dict_values)
omega = EeV_s * 10**5 /2
omega0 = (om2**0.5) * 10**5 /2

f1, f2 = chi_dct[str(EeV_s)].values()
X1 = (-f1/Z )* (omega0/omega)**2

In [19]:
# func and get x/y func
def get_y(theta, gamma, x1):
    return ALPHA/(PI**2) * theta ** 2 * (1/(theta** 2 + 1/(gamma**2)) - 1/(theta**2 + 1/(gamma**2) - x1))**2


def get_XY_for_plot(theta_cnt, gamma, x1, min_theta=-0.02, max_theta=0.02):
    x = np.linspace(min_theta, max_theta, theta_cnt )

    y = np.copy(x)
    for idx, theta in enumerate(x):
        y[idx] = get_y(theta, gamma, x1)
    return x, y


In [20]:
# graph
x_1, y_1 = get_XY_for_plot(1000, 300, X1)
x_2, y_2 = get_XY_for_plot(1000, 400, X1)


df = pd.DataFrame({
    "theta": np.concatenate([x_1, x_2]),
    "y": np.concatenate([y_1, y_2]),
    "label": ["График 1"] * len(x_1) + ["График 2"] * len(x_2)
})

fig = px.line(df, x="theta", y="y", color="label")


fig.update_layout(
    template="plotly_white"
)
fig.show()